In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)




# size of the window for data preparation
split_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list, x_temp_list, y_temp_list = [], [], [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - split_window_size, split_window_size + 1):
    x_train_temp = x_data.iloc[i:i+split_window_size+1]
    y_train_temp = y_data.iloc[i:i+split_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_temp = x_train_temp.iloc[-1:]
    y_temp = y_train_temp.iloc[-1:]

    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_temp_list.append(x_temp)
    y_temp_list.append(y_temp)

# Concatenate the lists into pandas DataFrames
x_train = pd.concat(x_train_list)
y_train = pd.concat(y_train_list)
x_temp = pd.concat(x_temp_list)
y_temp = pd.concat(y_temp_list)

# print(y_train.head(50))
x_temp_train, x_temp_val, y_temp_train, y_temp_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)


# Split x_temp and y_temp into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42)


"""
# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))
"""




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)




"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""



'\n# x_train_tensor inverse\n\nx_test_original = scaler.inverse_transform(x_train_tensor.numpy())\nprint("\nFirst row of x_test_original:")\nprint(x_test_original[0])\n\nprint("\nFirst row of x_train:")\nprint(x_train.head(1))\n\n\n\nprint("\nLast row of x_test_original:")\nprint(x_test_original[-1])\n\nprint("\nLast row of x_train:")\nprint(x_train.tail(1))\n'

In [9]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [10]:
import torch
import torch.nn as nn

window_size = 1

input_size = 7
hidden_size = 16
num_layers = 4

learning_rate = 0.001



num_epochs = 300


model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/300], Training Loss: 17726.996224456307, Validation Loss: 18998.43359375
Epoch [2/300], Training Loss: 14284.579095399544, Validation Loss: 18811.296875
Epoch [3/300], Training Loss: 10768.255800372946, Validation Loss: 21414.986328125
Epoch [4/300], Training Loss: 8097.40742481122, Validation Loss: 14617.8974609375
Epoch [5/300], Training Loss: 6205.981303983779, Validation Loss: 11536.42578125
Epoch [6/300], Training Loss: 4731.369281281198, Validation Loss: 9215.7255859375
Epoch [7/300], Training Loss: 3631.127809656652, Validation Loss: 6568.2275390625
Epoch [8/300], Training Loss: 2767.7662652004547, Validation Loss: 4199.408203125
Epoch [9/300], Training Loss: 2140.6312790858997, Validation Loss: 3179.010009765625
Epoch [10/300], Training Loss: 1679.440478884109, Validation Loss: 2725.8974609375
Epoch [11/300], Training Loss: 1338.2858870762457, Validation Loss: 2168.396240234375
Epoch [12/300], Training Loss: 1085.3877932939624, Validation Loss: 1652.029052734375
Epoch 

In [11]:
import itertools

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [16]
num_layers_list = [ 4]
learning_rates = [ 0.001]
window_sizes = [1, 2, 5, 10, 15, 20, 25, 30]
num_epochs = 80

# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

best_validation_loss = float('inf')
best_test_loss = float('inf')
best_hyperparameters = None

# Iterate over each hyperparameter combination
for hyperparameters in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparameters

    print()
    print("Hyperparameters:", hyperparameters)

    # Initialize the model with current hyperparameters
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validate the model
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

            avg_train_loss = running_loss / len(x_train_tensor)
            avg_val_loss = val_loss / len(x_val_tensor)

            print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')

    # Evaluate the model on test data
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    avg_test_loss = test_loss / len(x_test_tensor)
    print(f'Test Loss: {avg_test_loss}')

    # Update best hyperparameters if test loss is lower
    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        best_hyperparameters = hyperparameters

print()
print("Best Hyperparameters:", best_hyperparameters)
print("Best Test Loss:", best_test_loss)



Hyperparameters: (7, 16, 4, 0.001, 1)
Epoch [1/80], Training Loss: 17726.68072065603, Validation Loss: 18998.291015625
Epoch [2/80], Training Loss: 14268.656376043677, Validation Loss: 18815.806640625
Epoch [3/80], Training Loss: 10665.354157519032, Validation Loss: 21263.16015625
Epoch [4/80], Training Loss: 8875.495716726904, Validation Loss: 15064.1650390625
Epoch [5/80], Training Loss: 7058.00363547739, Validation Loss: 14003.8388671875
Epoch [6/80], Training Loss: 4901.483497820451, Validation Loss: 9878.9482421875
Epoch [7/80], Training Loss: 3714.9472711282474, Validation Loss: 6492.92578125
Epoch [8/80], Training Loss: 2806.323359056444, Validation Loss: 5080.19970703125
Epoch [9/80], Training Loss: 2170.3582344762513, Validation Loss: 3919.066650390625
Epoch [10/80], Training Loss: 1708.2795731893757, Validation Loss: 2608.429931640625
Epoch [11/80], Training Loss: 1364.3270508295295, Validation Loss: 2179.731201171875
Epoch [12/80], Training Loss: 1118.659401967779, Validati